# How to import Abaqus meshes

This notebook demonstrates how to import simulation results from Abaqus in pylife, e.g., to be used for a FKM nonlinear assessment.

For this notebook to run, you need an installation of Abaqus and you have to install the odbclient and odb-server.
Please refer to https://pylife.readthedocs.io/en/latest/tools/odbclient/

In [ ]:
import odbclient as CL
import numpy as np
import timeit

# pylife
import pylife
import pylife.vmap
import pylife.stress.equistress
import pylife.mesh.gradient

In [ ]:
### kt2-specimen, simplex mesh, 973490 nodes

## Select the location where Abaqus is installed
# The example file needs Abaqus 2020

#abaqus_bin = r"C:\Program Files\SIMULIA\2019\AbaqusCAE\win_b64\code\bin\ABQLauncher.exe"
abaqus_bin = r"C:\Program Files\SIMULIA\2020\EstProducts\win_b64\code\bin\ABQLauncher.exe"
#abaqus_bin = r"C:\Program Files\SIMULIA\2021\EstProducts\win_b64\code\bin\ABQLauncher.exe"

# load the mesh file
filename = 'data/kt1.odb'
client = CL.OdbClient(filename, abaqus_bin=abaqus_bin)

# Select and print the available instances, steps and frame_ids
instance = 'KT2-1'
instance = 'KT1-1'
step = 'Step-1'
increment = 1

print(f"instances: {client.instance_names()}")
print(f"steps: {client.step_names()}")
print(f"frames: {client.frame_ids(step)}")

variable_names = client.variable_names(step, increment)
print(f"variable names: {variable_names}")

# get the node coordinates
coordinates = client.node_coordinates(instance)

# select the stress variable
pylife_mesh = client.variable('S', instance, step, increment)

# calculate the Mises equistress
pylife_mesh['mises'] = pylife_mesh.equistress.mises()

# add the node coordinates and organize the dataframe multi-index
pylife_mesh = coordinates.join(pylife_mesh)
pylife_mesh = pylife_mesh.reorder_levels(["element_id", "node_id"])


In [ ]:
# Calculate the stress gradient (method 1)
tstart = timeit.default_timer()
grad = pylife_mesh.gradient.gradient_of('mises')
tend = timeit.default_timer()
print(f"duration calculate stress gradient: {tend-tstart:.1f} s")

grad["abs_grad"] = np.linalg.norm(grad, axis=1)

In [ ]:
# Calculate the stress gradient (method 2)
tstart = timeit.default_timer()
grad1 = pylife_mesh.gradient_3D.gradient_of('mises')
tend = timeit.default_timer()
print(f"duration calculate stress gradient: {tend-tstart:.1f} s")

grad1["abs_grad"] = np.linalg.norm(grad1, axis=1)
pylife_mesh = pylife_mesh.join(grad1, sort=False)

Now we have successfully imported a finite element mesh from an Abaqus simulation and calculated the Mises stress and the stress gradient.

In [ ]:
pylife_mesh

We can count the number of nodes for every element. This gives a hint about the element type.
* abaqus triangular: 10 nodes per element
* abaqus hex: 8 nodes per element
* (ansys hex: 20 nodes per element)

In [ ]:
pylife_mesh.groupby("element_id")[pylife_mesh.columns[0]].count()

We can also filter the mesh or extract a single element, e.g., element ID 1:

In [ ]:
pylife_mesh[pylife_mesh.index.get_level_values("element_id")==1]


Next, we plot the mesh using the pyvista package. The coloring corresponds to the absolute Mises stress.

In [ ]:
import pyvista as pv
grid = pv.UnstructuredGrid(*pylife_mesh.mesh.vtk_data())
plotter = pv.Plotter(window_size=[1920, 1080])
plotter.add_mesh(grid, scalars=pylife_mesh.groupby('element_id')['mises'].mean().to_numpy(),
                show_edges=True, cmap='jet')
plotter.add_scalar_bar()
plotter.show()